In [1]:
import sys
sys.path.append("..")

import logging
import random
import neptune
import numpy as np
import syft as sy
import torch

from torchvision import transforms
from federated_learning.FLCustomDataset import FLCustomDataset
from federated_learning.FederatedLearning import FederatedLearning
from federated_learning.helper import utils

CONFIG_PATH = '../configs/defaults.yml'
configs = utils.load_config(CONFIG_PATH)
arguments = dict()
arguments['--reg'] = 0.0
arguments['--output-prefix'] = "mnist_no_attack"
arguments['--server_model'] = "data_output/20201107_030649_mnist_w0/server_model_9"
arguments["--epoch"] = 1
arguments["--round"] = 1
output_dir = None
log_enable = False
neptune_enable = False


configs = utils.load_config(CONFIG_PATH)
logging.basicConfig(format='%(asctime)s %(message)s', level=configs['log']['level'])
random.seed(configs['runtime']['random_seed'])

# From command line
epochs_num = int(arguments["--epoch"])
rounds_num = int(arguments["--round"])


fl = FederatedLearning(
    configs['runtime']['batch_size'], 
    configs['runtime']['test_batch_size'], 
    configs['runtime']['lr'], 
    float(arguments['--reg']) if arguments['--reg'] is not None else 0.0,
    configs['runtime']['momentum'], 
    neptune_enable, log_enable, 
    configs['log']['interval'], 
    output_dir, 
    configs['runtime']['random_seed'])

total_num_workers = \
    configs['runtime']['mnist_normal_users_num'] + \
    configs['runtime']['mnist_eavesdropper_num'] + \
    configs['runtime']['mnist_trusted_users_num']

workers_idx = ["worker_" + str(i) for i in range(total_num_workers)]

fl.create_server()
fl.create_server_model()

trusted_idx = utils.get_workers_idx(
    range(total_num_workers), configs['runtime']['mnist_trusted_users_num'], [])
eavesdroppers_idx = utils.get_workers_idx(
    range(total_num_workers), configs['runtime']['mnist_eavesdropper_num'], trusted_idx)
normal_idx = utils.get_workers_idx(
    range(total_num_workers), configs['runtime']['mnist_normal_users_num'], trusted_idx + eavesdroppers_idx)

trusted_idx = [workers_idx[ii] for ii in trusted_idx]
eavesdroppers_idx = [workers_idx[ii] for ii in eavesdroppers_idx]
normal_idx = [workers_idx[ii] for ii in normal_idx]

logging.info("Trusted: {}".format(trusted_idx))
logging.info("Eavesdroppers: {}".format(eavesdroppers_idx))
logging.info("Normal: {}".format(normal_idx))

fl.create_workers(workers_idx)
fl.create_workers_model(workers_idx)

# train_raw_dataset = utils.preprocess_mnist(
#     utils.load_mnist_data_train(
#         configs['data']['MNIST_PATH'], 
#         configs['runtime']['mnist_data_percentage']))
# train_dataset = utils.get_mnist_dataset(train_raw_dataset)

# test_data = utils.load_mnist_data_test(configs['data']['MNIST_PATH'])
# test_dataset = utils.get_mnist_dataset(test_data)
# test_dataloader = utils.get_dataloader(
#     test_dataset, configs['runtime']['test_batch_size'], shuffle=True, drop_last=False)

# federated_train_dataloader = None
# arguments["--no-attack"] = False
# arguments["--attack"] = 1
# if arguments["--no-attack"]:
#     logging.info("No Attack will be performed.")
#     federated_train_dataloader = fl.create_federated_mnist(
#         train_dataset, workers_idx, configs['runtime']['batch_size'], shuffle=False)
# else:
#     logging.info("Perform attack type: {}".format(arguments["--attack"]))
#     federated_train_dataloader = fl.create_federated_mnist(
#         utils.perfrom_attack(
#             train_dataset, 
#             int(arguments["--attack"]), 
#             workers_idx, 
#             eavesdroppers_idx, 
#             100), 
#         workers_idx, configs['runtime']['batch_size'], shuffle=False)

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/home/ubuntu/.local/lib/python3.6/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'


2020-11-17 16:41:19,927 Initializing Federated Learning class...
2020-11-17 16:41:20,806 Creating the server...
2020-11-17 16:41:20,807 Creating a model for the server...
2020-11-17 16:41:20,817 Trusted: ['worker_26', 'worker_0', 'worker_19', 'worker_23', 'worker_12', 'worker_17', 'worker_36', 'worker_27', 'worker_10', 'worker_7']
2020-11-17 16:41:20,818 Eavesdroppers: ['worker_16', 'worker_35', 'worker_11', 'worker_39', 'worker_22']
2020-11-17 16:41:20,820 Normal: ['worker_5', 'worker_33', 'worker_37', 'worker_32', 'worker_9', 'worker_13', 'worker_4', 'worker_21', 'worker_20', 'worker_1', 'worker_29', 'worker_18', 'worker_6', 'worker_24', 'worker_14', 'worker_38', 'worker_2', 'worker_31', 'worker_8', 'worker_34', 'worker_3', 'worker_28', 'worker_30', 'worker_25', 'worker_15']
2020-11-17 16:41:20,821 Creating workers...
2020-11-17 16:41:20,824 Creating a model for 40 worker(s)...


In [2]:
fl.find_best_weights_from_trusted_idx(None, workers_idx, trusted_idx)

2020-11-17 16:42:40,280 Finding an average model for 10 workers.
2020-11-17 16:42:40,317 Start the optimization....
2020-11-17 16:43:50,885 [0.03369591 0.0308968  0.03328874 0.03282095 0.03395948 0.03693701
 0.03045136 0.03254906 0.03177949 0.03213917 0.03236526 0.03123167
 0.03366703 0.030027   0.0322833  0.03348121 0.03474772 0.02974499
 0.03214497 0.03394073 0.03373389 0.02932797 0.04127564 0.03373694
 0.03140149 0.03800234 0.03344242 0.03789577 0.03431675 0.03471492]
2020-11-17 16:43:50,888 


array([0.03369591, 0.0308968 , 0.03328874, 0.03282095, 0.03395948,
       0.03693701, 0.03045136, 0.03254906, 0.03177949, 0.03213917,
       0.03236526, 0.03123167, 0.03366703, 0.030027  , 0.0322833 ,
       0.03348121, 0.03474772, 0.02974499, 0.03214497, 0.03394073,
       0.03373389, 0.02932797, 0.04127564, 0.03373694, 0.03140149,
       0.03800234, 0.03344242, 0.03789577, 0.03431675, 0.03471492])